# Data Preparation

This is the **second** notebook of the analysis.

> Make sure you have executed the previous one [1_Explore.ipynb](1_Explore.ipynb), otherwise the packages might fail.
---

# Objective

After exploring the data. The following cells will prepare the dataset in a way that it could be consumed by a model.

At the end, a dataset will be pickled into a compressed file, which will be later used in the next processes (Train and Forecast)